**CAPSTONE PROJECT**

with key usage
1. Agent
2. Agent Tool
3. Sessions & Memory

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [2]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, AgentTool, ToolContext , load_memory, preload_memory
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
from pydantic import BaseModel, Field
laporan_database = {} # Dictionary untuk menyimpan laporan dengan ID sebagai kunci
# --- Tool 1: Pencatatan Laporan ---
class LaporanInput(BaseModel):
    """Skema input untuk mencatat laporan keluhan warga."""
    judul: str = Field(description="Judul singkat atau ringkasan keluhan.")
    deskripsi: str = Field(description="Detail lengkap dan lokasi kejadian keluhan.")
    kategori: str = Field(description="Kategori keluhan (ex: Infrastruktur, Lingkungan, Administrasi).")


def catat_laporan(judul: str, deskripsi: str, kategori: str, urgensi: str) -> str:
    """Mencatat laporan baru ke dalam sistem basis data (simulasi)."""
    laporan_id = f"LAP-{hash(judul + deskripsi) % 10000}" # ID yang lebih unik
    laporan_database[laporan_id] = {
        "judul": judul,
        "deskripsi": deskripsi,
        "kategori": kategori,
        "urgensi": urgensi,
        "status": "Diterima", # Status awal
        "tanggal_lapor": "2023-10-27" # Tanggal simulasi
    }
    print(f"\n[TOOL LOG: catat_laporan] Laporan telah dicatat.")
    print(f"ID: {laporan_id}, Kategori: {kategori}, Urgensi: {urgensi}, Status: {laporan_database[laporan_id]['status']}")
    return f"Laporan Anda (ID: {laporan_id}) berhasil dicatat dengan kategori {kategori} dan tingkat urgensi *{urgensi}*. Status awal: {laporan_database[laporan_id]['status']}. Kami akan segera memprosesnya."
# --- Tool 2: Penentuan Unit Penanggung Jawab ---

def tentukan_unit_penanggung_jawab(kategori_laporan: str) -> str:
    """Menentukan dinas atau unit pemerintah yang bertanggung jawab berdasarkan kategori."""

    # 1. Normalisasi Input Agen: Ubah ke huruf kecil dan hapus spasi ekstra
    kategori_norm = kategori_laporan.lower().strip()

    # 2. Definisikan mapping dengan kunci huruf kecil
    mapping = {
        "infrastruktur jalan": "Dinas Pekerjaan Umum (DPU)",
        "lingkungan": "Dinas Lingkungan Hidup (DLH)",
        "administrasi": "Dinas Kependudukan dan Pencatatan Sipil (Dukcapil)",
        "administrasi kependudukan": "Bagian Pelayanan Terpadu Satu Pintu (PTSP)",
        "lainnya": "Sekretariat Daerah"
    }


    unit = mapping.get(kategori_norm, "Sekretariat Daerah")

    return unit
def check_urgency_level(deskripsi_laporan: str) -> str:
    """Mengklasifikasikan urgensi laporan (Tinggi, Sedang, Rendah) berdasarkan kata kunci."""

    deskripsi_lower = deskripsi_laporan.lower()


    if any(keyword in deskripsi_lower for keyword in ["berbahaya", "ancaman", "segera", "darurat", "kebakaran", "ambruk", "kritis", "kecelakaan"]):
        urgensi = "Tinggi"
    elif any(keyword in deskripsi_lower for keyword in ["terlambat", "tidak berfungsi", "rusak", "mengganggu", "lama"]):
        urgensi = "Sedang"
    else:
        urgensi = "Rendah"

    return urgensi
def verify_incident_location(lokasi_string: str) -> str:
    """
    Memverifikasi dan menstandardisasi string lokasi insiden.
    Mensimulasikan pemanggilan API Geocoding untuk mendapatkan koordinat.
    """
    lokasi_lower = lokasi_string.lower()

    # Simulasi database lokasi yang valid dan hasilnya
    valid_locations = {
        "depan kantor walikota jakarta pusat": "Jl. Medan Merdeka Selatan No.8-9, Gambir, Jakarta Pusat (Lat: -6.17511, Lon: 106.82708)",
        "jl. sudirman": "Jl. Jenderal Sudirman, Jakarta (Lat: -6.20876, Lon: 106.82058)",
        "perempatan utama": "Perempatan Thamrin-Sudirman, Jakarta (Lat: -6.1952, Lon: 106.8227)",
        "kantor bupati bandung": "Jl. Raya Soreang-Ciwidey KM. 17, Soreang, Bandung (Lat: -7.0098, Lon: 107.5199)",
        "alun-alun kota surabaya": "Jl. Gubernur Suryo No.15, Surabaya (Lat: -7.2604, Lon: 112.7410)"
    }

    # Cari lokasi yang cocok
    for key, value in valid_locations.items():
        if key in lokasi_lower:
            print(f"[TOOL LOG: verify_incident_location] Lokasi ditemukan & diverifikasi: {value}")
            return value

    print(f"[TOOL LOG: verify_incident_location] Lokasi tidak dapat diverifikasi secara akurat: {lokasi_string}")
    return f"Lokasi tidak dapat diverifikasi secara akurat: '{lokasi_string}'. Mohon berikan detail lebih lanjut."
def search_knowledge_base(query: str) -> str:
    """
    Mencari basis data pengetahuan pemerintah untuk menemukan informasi, FAQ,
    atau panduan terkait query yang diberikan.
    """
    # Simulasi basis data pengetahuan. Dalam implementasi nyata, ini akan memanggil API DB
    knowledge_db = {
        "akta kelahiran": "Untuk mengurus akta kelahiran, siapkan KTP orang tua, KK, surat keterangan lahir dari RS/Bidan, dan surat nikah. Kunjungi kantor Dukcapil terdekat.",
        "jalan berlubang": "Laporan jalan berlubang akan diteruskan ke Dinas Pekerjaan Umum (DPU). Estimasi perbaikan tergantung tingkat kerusakan dan ketersediaan anggaran. Harap bersabar.",
        "sampah menumpuk": "Keluhan sampah menumpuk akan ditangani oleh Dinas Lingkungan Hidup (DLH). Pastikan membuang sampah pada tempat dan jadwal yang ditentukan.",
        "izin usaha": "Prosedur izin usaha dapat diakses melalui portal OSS (Online Single Submission). Siapkan NIK, NPWP, dan data usaha Anda.",
        "bpjs": "Informasi BPJS terkait pendaftaran, cek status, dan pembayaran iuran dapat diakses melalui aplikasi Mobile JKN atau website BPJS Kesehatan."
    }

    query_lower = query.lower()
    
    # Mencari query di basis data pengetahuan
    for keyword, info in knowledge_db.items():
        if keyword in query_lower:
            print(f"[TOOL LOG: search_knowledge_base] Menemukan informasi terkait '{keyword}'.")
            return f"Informasi yang ditemukan terkait '{keyword}': {info}"
    
    print(f"[TOOL LOG: search_knowledge_base] Tidak ada informasi langsung ditemukan untuk query: '{query}'.")
    return "Tidak ada informasi yang relevan ditemukan dalam basis data pengetahuan kami untuk pertanyaan Anda."
def monitor_and_update_status(laporan_id: str, new_status: str) -> str:
    """
    Memantau atau memperbarui status laporan berdasarkan ID Laporan.
    Jika new_status diberikan, akan memperbarui status laporan.
    """
    if laporan_id not in laporan_database:
        print(f"[TOOL LOG: monitor_and_update_status] ID Laporan '{laporan_id}' tidak ditemukan.")
        return f"ID Laporan '{laporan_id}' tidak ditemukan dalam sistem kami."

    current_laporan = laporan_database[laporan_id]

    if new_status:
        # Validasi status baru (opsional, bisa lebih kompleks)
        valid_statuses = ["Diterima", "Dalam Peninjauan", "Sedang Ditangani", "Selesai", "Dibatalkan"]
        if new_status not in valid_statuses:
            return f"Status '{new_status}' tidak valid. Status yang tersedia: {', '.join(valid_statuses)}."
        
        current_laporan["status"] = new_status
        print(f"[TOOL LOG: monitor_and_update_status] Status Laporan ID '{laporan_id}' diperbarui menjadi '{new_status}'.")
        return f"Status laporan ID '{laporan_id}' berhasil diperbarui menjadi *{new_status}*. Detail: {current_laporan}"
    else:
        print(f"[TOOL LOG: monitor_and_update_status] Memeriksa status Laporan ID '{laporan_id}'.")
        return f"Status laporan ID '{laporan_id}' saat ini adalah *{current_laporan['status']}*. Detail: {current_laporan}"

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [5]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [6]:
# Define constants used throughout the notebook
APP_NAME = "Agen_Pelaporan_dan_Keluhan_Warga"
USER_ID = "demo_user"

memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing


complaint_agent = LlmAgent(
    name="Agen_Pelaporan_dan_Keluhan_Warga",
    description="Agen AI untuk membantu warga mencatat keluhan, mengkategorikannya, dan mengarahkan ke unit penanggung jawab yang tepat.",
    tools=[catat_laporan, 
    tentukan_unit_penanggung_jawab, 
    check_urgency_level, 
    search_knowledge_base,
    monitor_and_update_status,
    load_memory],
    instruction="""
    Anda adalah Agen Pelaporan dan Keluhan Pemerintah yang profesional.
    Tugas Anda adalah:
    1.  Jika pengguna bertanya informasi umum atau prosedur, Wajib MENGGUNAKAN TOOL 'search_knowledge_base' terlebih dahulu.
    2.  Ekstrak deskripsi dan lokasi laporan dari pengguna.
    3.  Wajib MENGGUNAKAN TOOL 'verify_incident_location' dengan lokasi yang diekstrak.
    4.  Wajib MENGGUNAKAN TOOL 'check_urgency_level' untuk menentukan tingkat Urgensi (Tinggi, Sedang, Rendah).
    5.  Wajib MENGGUNAKAN TOOL 'tentukan_unit_penanggung_jawab' untuk menentukan dinas terkait.
    6.  Setelah memiliki Judul, Deskripsi, Kategori, dan Urgensi, Wajib MENGGUNAKAN TOOL 'catat_laporan'.
    7. Tentukan kategori laporan secara internal (misal: Infrastruktur, Lingkungan, Administrasi).
    8. Jawab pertanyaan pengguna dengan kata-kata sederhana. Gunakan alat load_memory jika Anda perlu mengingat percakapan sebelumnya.
    9.  Berikan konfirmasi yang ramah, Urgensi yang terdeteksi, dan ID laporan kepada pengguna.
    """,

    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config)
)



print("Agen Pelaporan dan Keluhan berhasil dibuat.")

Agen Pelaporan dan Keluhan berhasil dibuat.


In [7]:
# Create Session Service
session_service = InMemorySessionService()  # Handles conversations
# Create runner with BOTH services
runner = Runner(
    agent=complaint_agent,
     app_name="Agen_Pelaporan_dan_Keluhan_Warga",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)


In [8]:
await run_session(
    runner,
    "Saya mau lapor, ada kabel listrik putus menjuntai ke jalan di perempatan utama Thamrin-Sudirman. Sangat berbahaya dan bisa menyebabkan kecelakaan!",
    "conversation-01",  # Session ID
)
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)
await memory_service.add_session_to_memory(session)

print("✅ Session added to memory!")


### Session: conversation-01

User > Saya mau lapor, ada kabel listrik putus menjuntai ke jalan di perempatan utama Thamrin-Sudirman. Sangat berbahaya dan bisa menyebabkan kecelakaan!



[TOOL LOG: catat_laporan] Laporan telah dicatat.
ID: LAP-1200, Kategori: Infrastruktur, Urgensi: Tinggi, Status: Diterima


Model: > Terima kasih atas laporan Anda. Laporan mengenai "Kabel Listrik Putus di Thamrin-Sudirman" telah berhasil kami catat dengan tingkat urgensi **Tinggi**. Nomor laporan Anda adalah **LAP-1200**. Kami akan segera menindaklanjuti laporan ini.
✅ Session added to memory!


In [9]:
#test 2
await run_session(
    runner,
    "Saya ingin mengajukan komplain karena proses pembuatan akta kelahiran anak saya mengalami keterlambatan lebih dari batas waktu yang dijanjikan yaitu 14 hari jam kerja. Lokasinya di sekitar kantor walikota Jakarta Pusat.",
    "conversation-02",  # Session ID
)
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-02"
)
await memory_service.add_session_to_memory(session)

print("✅ Session added to memory!")


### Session: conversation-02

User > Saya ingin mengajukan komplain karena proses pembuatan akta kelahiran anak saya mengalami keterlambatan lebih dari batas waktu yang dijanjikan yaitu 14 hari jam kerja. Lokasinya di sekitar kantor walikota Jakarta Pusat.



[TOOL LOG: catat_laporan] Laporan telah dicatat.
ID: LAP-1716, Kategori: Administrasi, Urgensi: Sedang, Status: Diterima


Model: > Baik, Bapak/Ibu. Saya telah mencatat laporan Anda mengenai keterlambatan proses pembuatan akta kelahiran anak Anda di sekitar kantor walikota Jakarta Pusat.

Tingkat urgensi laporan ini terdeteksi sebagai **Sedang**.

Laporan Anda berhasil dicatat dengan ID: **LAP-1716**. Dinas Kependudukan dan Pencatatan Sipil (Dukcapil) akan menjadi unit penanggung jawab untuk keluhan ini. Kami akan segera memprosesnya.
✅ Session added to memory!


In [10]:
#test 3
await run_session(
    runner,
    "Saya ingin tahu prosedur bpjs?",
    "conversation-03",  # Session ID
)



### Session: conversation-03

User > Saya ingin tahu prosedur bpjs?


[TOOL LOG: search_knowledge_base] Menemukan informasi terkait 'bpjs'.
Model: > Untuk prosedur BPJS, Anda bisa mengakses informasi terkait pendaftaran, cek status, dan pembayaran iuran melalui aplikasi Mobile JKN atau website resmi BPJS Kesehatan.


In [11]:
runner = Runner(
    agent=complaint_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(
    runner, "apa saja informasi yang tersimpan?", "conversation-01"  # Different session ID
)


### Session: conversation-01

User > apa saja informasi yang tersimpan?


Model: > Saya telah mencatat laporan Anda sebelumnya:

*   **Judul Laporan:** Kabel Listrik Putus di Thamrin-Sudirman
*   **Deskripsi:** Kabel listrik putus menjuntai ke jalan di perempatan utama Thamrin-Sudirman. Sangat berbahaya dan bisa menyebabkan kecelakaan!
*   **Kategori:** Infrastruktur
*   **Tingkat Urgensi:** Tinggi
*   **ID Laporan:** LAP-1200

Ada juga laporan lain yang tersimpan dalam ingatan saya mengenai keterlambatan proses pembuatan akta kelahiran anak dengan ID LAP-1716.


In [12]:
await run_session(
    runner, "Bagaimana status laporan saya dengan ID LAP-3001?", "conversation-01"  # Different session ID
)


### Session: conversation-01

User > Bagaimana status laporan saya dengan ID LAP-3001?


Model: > Mohon maaf, saya tidak dapat memeriksa status laporan LAP-3001 saat ini. Sistem kami membutuhkan informasi status terbaru yang ingin Anda perbarui. Apakah Anda ingin memperbarui status laporan ini? Jika tidak, saya tidak dapat mengambil statusnya tanpa informasi tersebut.


In [13]:
await run_session(
    runner, "Sebagai Admin, saya ingin memperbarui status laporan LAP-3001 menjadi Sedang Ditangani.", "conversation-01"  # Different session ID
)


### Session: conversation-01

User > Sebagai Admin, saya ingin memperbarui status laporan LAP-3001 menjadi Sedang Ditangani.


[TOOL LOG: monitor_and_update_status] ID Laporan 'LAP-3001' tidak ditemukan.
Model: > Mohon maaf, ID laporan **LAP-3001** tidak ditemukan dalam sistem kami. Apakah Anda yakin dengan ID laporan tersebut?


In [14]:
await run_session(
    runner, "Bagaimana status laporan saya dengan ID LAP-3001", "conversation-01"  # Different session ID
)


### Session: conversation-01

User > Bagaimana status laporan saya dengan ID LAP-3001


Model: > Mohon maaf, ID laporan **LAP-3001** tidak ditemukan dalam sistem kami. Sepertinya laporan dengan ID tersebut belum tercatat.
